In [4]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [5]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [6]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='/home/hasu/Documents/github/content/MNIST/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='/home/hasu/Documents/github/content/MNIST/',
                                          train=False, 
                                          transform=transforms.ToTensor())

100%|█████████████████████████████| 9912422/9912422 [00:19<00:00, 498723.60it/s]


Extracting /home/hasu/Documents/github/content/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to /home/hasu/Documents/github/content/MNIST/MNIST/raw



100%|███████████████████████████████| 28881/28881 [00:00<00:00, 12780723.13it/s]

Extracting /home/hasu/Documents/github/content/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to /home/hasu/Documents/github/content/MNIST/MNIST/raw



100%|█████████████████████████████| 1648877/1648877 [00:03<00:00, 543349.15it/s]


Extracting /home/hasu/Documents/github/content/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/hasu/Documents/github/content/MNIST/MNIST/raw



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 6668018.47it/s]

Extracting /home/hasu/Documents/github/content/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/hasu/Documents/github/content/MNIST/MNIST/raw



In [7]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [10]:
len(train_loader)

600

In [11]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [12]:
model = ConvNet(num_classes).to(device)

In [13]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2743
Epoch [1/5], Step [200/600], Loss: 0.0989
Epoch [1/5], Step [300/600], Loss: 0.0367
Epoch [1/5], Step [400/600], Loss: 0.0888
Epoch [1/5], Step [500/600], Loss: 0.0663
Epoch [1/5], Step [600/600], Loss: 0.1026
Epoch [2/5], Step [100/600], Loss: 0.0227
Epoch [2/5], Step [200/600], Loss: 0.0173
Epoch [2/5], Step [300/600], Loss: 0.0473
Epoch [2/5], Step [400/600], Loss: 0.0381
Epoch [2/5], Step [500/600], Loss: 0.1040
Epoch [2/5], Step [600/600], Loss: 0.0274
Epoch [3/5], Step [100/600], Loss: 0.0296
Epoch [3/5], Step [200/600], Loss: 0.0169
Epoch [3/5], Step [300/600], Loss: 0.0776
Epoch [3/5], Step [400/600], Loss: 0.0078
Epoch [3/5], Step [500/600], Loss: 0.0322
Epoch [3/5], Step [600/600], Loss: 0.0604
Epoch [4/5], Step [100/600], Loss: 0.0274
Epoch [4/5], Step [200/600], Loss: 0.0129
Epoch [4/5], Step [300/600], Loss: 0.0955
Epoch [4/5], Step [400/600], Loss: 0.0029
Epoch [4/5], Step [500/600], Loss: 0.0155
Epoch [4/5], Step [600/600], Loss:

In [15]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.96 %


In [ ]:
# # Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')